#### NOTA: En este cuaderno se creara la Tabla `gasto` del Datawarehouse

***
### Importando Librerias

In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import count, when, isnull

***
Creando Session de Spark

In [3]:
spark = SparkSession.builder.appName('gasto').getOrCreate()

***
Importando Conexion a Contenedor `mod4_mysql`

In [4]:
from db.connection import connec_db

***
Trabajando el csv asociado a  `gasto`

In [4]:
gasto = spark.read.csv("Data/Gasto.csv", sep=",", header=True, inferSchema=True)

***
Visualizando como vienen los Datos

In [5]:
gasto.show(5)

+-------+----------+-----------+----------+-------+
|IdGasto|IdSucursal|IdTipoGasto|     Fecha|  Monto|
+-------+----------+-----------+----------+-------+
|      1|        18|          1|2015-01-01| 1164.9|
|      2|        18|          2|2015-01-01| 317.02|
|      3|        18|          3|2015-01-01| 118.89|
|      4|        18|          4|2015-01-01|1174.23|
|      5|         1|          1|2015-01-01|1104.51|
+-------+----------+-----------+----------+-------+
only showing top 5 rows



***
Visualizando la Inferencia del Schema

In [6]:
gasto.printSchema()

root
 |-- IdGasto: integer (nullable = true)
 |-- IdSucursal: integer (nullable = true)
 |-- IdTipoGasto: integer (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Monto: double (nullable = true)



***
Buscando Valores Faltantes

In [7]:
# Contar valores nulos por columna
gasto.agg(*[count(when(isnull(c), c)).alias(c) for c in gasto.columns]).show()



+-------+----------+-----------+-----+-----+
|IdGasto|IdSucursal|IdTipoGasto|Fecha|Monto|
+-------+----------+-----------+-----+-----+
|      0|         0|          0|    0|    0|
+-------+----------+-----------+-----+-----+



***
Creando la Conexion

In [6]:
engine = connec_db()

***
Pasando el Dataframe a Dataframe de Pandas  y vaciando en su respectiva Tabla

In [8]:
# IMPORTANDO LIBRERIA PANDAS
import pandas as pd

***
Vaciando el Dataframe a la Tabla `gasto`

In [9]:
pandas_df_gasto = gasto.toPandas()


In [9]:
pandas_df_gasto.to_sql(name="gasto", con=engine, index=False, if_exists="append")

8640

In [10]:
pandas_df_gasto.to_csv("csv_practica/gasto.csv", index=False)

In [11]:
spark.stop()